Take FCN, pass a sliding window over the input and output the produced annotation

In [1]:
import sys
sys.path.insert(0, '/data1/dbashir/Project/Summer2018/SheetIdentification/sheet_id/utils/')
sys.path.insert(0, '/data1/dbashir/Project/Summer2018/SheetIdentification/sheet_id/models/')

In [2]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint

from FCN import FCN
from loss_functions import softmax_sparse_crossentropy_ignoring_background
from metrics import sparse_accuracy_ignoring_background
from dataPreprocessing import splitTrainValidation
from dataGenerator import DataGenerator

Using TensorFlow backend.
/home/dbashir/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
model = FCN(input_shape=(250,250,1), n_classes=124)

In [4]:
loss_fn = softmax_sparse_crossentropy_ignoring_background
optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, 
                 epsilon=1e-8, decay=0.0, amsgrad=False)
metrics = [sparse_accuracy_ignoring_background]
model.compile(loss=loss_fn, optimizer=optimizer, metrics=metrics)

In [7]:
def toAnnotation(img, model):
    """
    takes an image and runs model over it
    """
    #img = tf.placeholder(tf.float32, shape=(1, None, None, 1))
    img_slides = tf.extract_image_patches(img,
                                          [1,250,250,1], 
                                          [1,1,1,1], 
                                          [1,1,1,1], 
                                          'VALID')

    x = K.reshape(img_slides, [-1,250,250,1])
    y = model(x)
    
    
    image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, None, interpolation=cv2.INTER_CUBIC)
    #image = image[0:500,0:500]
    
    img_shape = image.shape
    img_reshaped = image.reshape(1, img_shape[0], img_shape[1], 1)

    with tf.Session(config=config).as_default() as sess:
    #     run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
    #     sess = tf_debug.TensorBoardDebugWrapperSession(sess, "mirlab:6000")model
        #model.load_weights('noteheadDetector/models/cnn8.h5')
        output_vals = sess.run(y, {img: img_reshaped,
                                   K.learning_phase(): 0})

In [8]:
img = 'true_piece.png'

In [10]:
import tensorflow as tf
toAnnotation(img, model)

TypeError: Value passed to parameter 'images' has DataType string not in list of allowed values: float32, float64, int32, int64, uint8, int16, int8, uint16, float16